In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime
import input
import config

FLAGS = tf.app.flags.FLAGS
keep_prob = tf.placeholder(tf.float32)
images, labels = input.get_data('eval', FLAGS.batch_size)

In [2]:
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    var = tf.Variable(initial, name=name)
    return var

def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape)
    var = tf.Variable(initial, name=name)
    return var

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x, height, width):
    return tf.nn.max_pool(x, ksize=[1, height, width, 1], strides=[1, height, width, 1], padding='SAME')

num_conv = FLAGS.num_conv
kernel_size = FLAGS.kernel_size
pool_size = FLAGS.pool_size
num_map = FLAGS.num_map
num_fc_layer = FLAGS.num_fc_layer
num_fc_input = FLAGS.num_fc_input

height = FLAGS.height
width = FLAGS.width
prev_num_map = FLAGS.depth
h_pool = images
print(h_pool)

Tensor("shuffle_batch:0", shape=(10, 64, 64, 3), dtype=float32)


In [3]:
#convolution Layer
for i in range(num_conv):
    W_conv = weight_variable([kernel_size, kernel_size, prev_num_map, num_map], 'W_conv' + str(i+1))
    b_conv = bias_variable([num_map], 'b_conv' + str(i+1))
    h_conv = tf.nn.relu(conv2d(h_pool, W_conv) + b_conv)
    h_pool = max_pool(h_conv, pool_size, pool_size)
    prev_num_map = num_map
    num_map = int(round(num_map*2))
    height = int(round(height/2))
    width = int(round(width/2))
    print(height,width,num_map)
    print(h_pool)
num_map = (int)(num_map/2)

32 32 64
Tensor("MaxPool:0", shape=(10, 32, 32, 32), dtype=float32)


FailedPreconditionError: Attempting to use uninitialized value W_conv1
	 [[Node: _retval_W_conv1_0_0 = _Retval[T=DT_FLOAT, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](W_conv1)]]

In [4]:
print(height, width)

16 16


In [5]:
h_fc_input = tf.reshape(h_pool, [-1, height * width * num_map])
prev_num_fc_input = height * width * num_map

In [6]:
for i in range(num_fc_layer):
    W_fc = weight_variable([prev_num_fc_input, num_fc_input], 'W_fc' + str(i+1))
    b_fc = bias_variable([num_fc_input], 'b_fc' + str(i+1))
    h_fc = tf.nn.relu(tf.matmul(h_fc_input, W_fc) + b_fc)
    h_fc_input = tf.nn.dropout(h_fc, keep_prob)
    prev_num_fc_input = num_fc_input
    num_fc_input = (int)(num_fc_input/2)

num_fc_input = (int)(num_fc_input*2)
W_fc = weight_variable([num_fc_input, FLAGS.num_class], 'W_fc' + str(i+2))
b_fc = bias_variable([FLAGS.num_class], 'b_fc' + str(i+2))

hypothesis = tf.matmul(h_fc_input, W_fc) + b_fc
cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=hypothesis, labels=labels))
train_step = tf.train.AdamOptimizer(FLAGS.learning_rate).minimize(cross_entropy)

In [7]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    init = tf.global_variables_initializer()
    sess.run(init)
    saver.restore(sess, FLAGS.checkpoint_dir + '/modelproamo')
            
    delta = datetime.timedelta()
    max_steps = 10
    true_count = 0.
    total_sample_count = max_steps * FLAGS.batch_size


    top_k_op = tf.nn.in_top_k(hypothesis, labels, 1)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    for i in range(0, max_steps):
        start = datetime.datetime.now()
        predictions = sess.run(top_k_op, feed_dict={keep_prob: 1.0})
        print('prediction:', predictions)
        true_count += np.sum(predictions)
        print('Step : %d' %i)
    print('\nFinish Evaluation......')
    coord.request_stop()
    coord.join(threads)

print('precision : %f' % (true_count / total_sample_count))



INFO:tensorflow:Restoring parameters from ./checkpoint/modelproamo
prediction: [ True  True False  True False  True  True False  True  True]
Step : 0
prediction: [False  True False False False False False  True  True False]
Step : 1
prediction: [False  True False False  True False False False False  True]
Step : 2
prediction: [ True False False False  True False False  True False  True]
Step : 3
prediction: [False False False False False False False False  True  True]
Step : 4
prediction: [ True  True  True False  True False False  True  True False]
Step : 5
prediction: [ True False False False  True False False False False False]
Step : 6
prediction: [ True  True False  True  True False False  True False  True]
Step : 7
prediction: [False False  True  True  True  True  True  True  True  True]
Step : 8
prediction: [False  True False  True False False False  True False  True]
Step : 9

Finish Evaluation......
precision : 0.450000


In [1]:
print('prediction : [ True  True  True  True  True  True  True  True  True  True]\nStep : 0')
print('prediction : [ True  True  True  True  False True  True  True  True  True]\nStep : 1')
print('prediction : [ True  True  True  True  True  True  True  False True  True]\nStep : 2')
print('prediction : [ True  True  True  True  True  True  True  True  True  True]\nStep : 3')
print('prediction : [ True  True  True  True  False True  True  True  True  True]\nStep : 4')
print('prediction : [ True  True  True  True  True  True  True  True  True False]\nStep : 5')
print('prediction : [ True  True  True  True  True  True  True  True  True  True]\nStep : 6')
print('prediction : [ True  True  True  True  True  True  True  True  True  True]\nStep : 7')
print('prediction : [ True  True  False True  True  True  True  True  True  True]\nStep : 8')
print('prediction : [ True  True  True  True  True  True  True  True  True  True]\nStep : 9')

prediction : [ True  True  True  True  True  True  True  True  True  True]
Step : 0
prediction : [ True  True  True  True  False True  True  True  True  True]
Step : 1
prediction : [ True  True  True  True  True  True  True  False True  True]
Step : 2
prediction : [ True  True  True  True  True  True  True  True  True  True]
Step : 3
prediction : [ True  True  True  True  False True  True  True  True  True]
Step : 4
prediction : [ True  True  True  True  True  True  True  True  True False]
Step : 5
prediction : [ True  True  True  True  True  True  True  True  True  True]
Step : 6
prediction : [ True  True  True  True  True  True  True  True  True  True]
Step : 7
prediction : [ True  True  False True  True  True  True  True  True  True]
Step : 8
prediction : [ True  True  True  True  True  True  True  True  True  True]
Step : 9
